Import Libraries

In [ ]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url
from torch.utils.model_zoo import load_url as load_state_dict_from_url
from typing import Type, Any, Callable, Union, List, Dict, Optional, cast
from torch import Tensor
from collections import OrderedDict 

Build New Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
from PIL import Image
import cv2
import torchvision.transforms as transforms
import collections
import random
from random import seed, choice, sample
import os
import json
from google.colab.patches import cv2_imshow
import sys
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False
from os.path import exists, join, basename, splitext
import csv
import glob

In [ ]:
%matplotlib inline
import os
import sys
import json
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import skimage.transform
from PIL import Image
from collections import Counter
from tqdm.notebook import tqdm

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence
from nltk.translate.bleu_score import corpus_bleu

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

from gensim.models.fasttext import FastText
 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import torchvision.models as models

In [ ]:
os.chdir('/content/drive/My Drive/Final Year Project/')

In [ ]:
def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id={file_id}" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm={confirm_text}&id={file_id}" -o {file_name} 

In [ ]:
def unique_list(words, confs):
  unique_list_words = []
  unique_list_confs = []
  for i in range(0, len(words)):
    if words[i] not in unique_list_words:
      unique_list_words.append(words[i])
      unique_list_confs.append(confs[i])
  return unique_list_words, unique_list_confs

In [ ]:
print(os.path.abspath('.'))

/content/drive/.shortcut-targets-by-id/1sz5w7NJWd5pUCRXsoinjyyZ-EUMh1tbM/Final Year Project


In [ ]:
json_path = '/content/drive/My Drive/Final Year Project/'
annotation_folder = json_path + '/dataset/annotations/'

max_len = 100               # length of padded captions
min_word_freq = 5           # words occuring less than this threshold will be labelled as <unk>
captions_per_image = 5      # number of captions per image
word_freq = Counter()       # Counter object to find the freq of each word


def to_lower(caption_words):
  for i in range(len(caption_words)):
    word = caption_words[i]
    word = word.lower()
    caption_words[i] = word
  return caption_words

count = 0
train_captions_json_path = annotation_folder + 'train.json'
with open(train_captions_json_path, 'r') as f:
    data_train = json.load(f)
image_folder_train = json_path + 'dataset/train/'   # folder with download images for train set
for val in data_train['annotations']:
  #print(val)
  path = image_folder_train + 'VizWiz_train_' + '%08d.jpg' % (val['image_id'])
  count = count+1
  caption = val['caption']
  caption_words = caption.split()
  caption_words = to_lower(caption_words)
  word_freq.update(caption_words)
  #print(word_freq.keys())
  #print(caption_words)
  #print(count)
print("Count of train images' captions encoded: ",count)

count=0
val_captions_json_path = annotation_folder + 'val.json'
with open(val_captions_json_path, 'r') as f:
    data_val = json.load(f)
image_folder_val = json_path + 'dataset/val/'   # folder with download images for val set
for val in data_val['annotations']:
  #print(val)
  path = image_folder_val + 'VizWiz_val_' + '%08d.jpg' % (val['image_id']-23431)
  #print((val['image_id']-23431))
  count = count+1
  caption = val['caption']
  caption_words = caption.split()
  caption_words = to_lower(caption_words)
  word_freq.update(caption_words)
  #print(word_freq.keys())
  #print(caption_words)
  #print(count)
print("Count of val images' captions encoded: ",count)

# create word map (dict mapping words to ids with 0 reserved for padding)
words = [word for word in word_freq.keys() if word_freq[word] > min_word_freq]
word2id = {word: id for id, word in enumerate(words, 1)}
word2id['<unk>'] = len(word2id) + 1
word2id['<start>'] = len(word2id) + 1
word2id['<end>'] = len(word2id) + 1
word2id['<pad>'] = 0

# create reverse word map (dict mapping ids to words)
id2word = {value: word for word, value in word2id.items()}
print(id2word)

Count of train images' captions encoded:  117155
Count of val images' captions encoded:  38750
{1: 'its', 2: 'is', 3: 'a', 4: 'basil', 5: 'leaves', 6: 'container', 7: 'contains', 8: 'the', 9: 'net', 10: 'weight', 11: 'too.', 12: 'green', 13: 'and', 14: 'white', 15: 'plastic', 16: 'condiment', 17: 'bottle', 18: 'containing', 19: 'leaves.', 20: 'quality', 21: 'issues', 22: 'are', 23: 'too', 24: 'severe', 25: 'to', 26: 'recognize', 27: 'visual', 28: 'content.', 29: 'of', 30: 'spices', 31: 'in', 32: 'laying', 33: 'on', 34: 'surface.', 35: 'some', 36: 'counter', 37: 'can', 38: 'coca', 39: 'cola', 40: 'shown', 41: 'for', 42: 'when', 43: 'one', 44: 'use', 45: 'cold', 46: 'drink.', 47: 'black', 48: 'zero', 49: 'calorie', 50: 'soda', 51: 'near', 52: 'coffee', 53: 'maker.', 54: 'kitchen', 55: 'various', 56: 'items', 57: 'top', 58: 'including', 59: 'metal', 60: 'containers,', 61: 'tin', 62: 'placed', 63: 'surface', 64: 'with', 65: 'kettle', 66: 'soda.', 67: 'crushed', 68: 'tomatoes', 69: 'brown',

In [ ]:
# split the images and captions into train and val sets
train_image_paths = []
train_image_captions = []
train_caption_lens = []

split = "train"
count = 0
captions = []
for val in data_train['annotations']:
  #print(val)
  path = image_folder_train + 'VizWiz_train_' + '%08d.jpg' % (val['image_id'])
  count = count+1
  caption = val['caption']
  #print(caption)
  caption_words = caption.split()
  for i in range(len(caption_words)):
    word = caption_words[i]
    word = word.lower()
    caption_words[i] = word
  if(len(caption_words) <= max_len):
    captions.append(caption_words)
    
  if(len(captions) == 0):
    continue
    
  # sample the captions randomly such that every image has same no of captions
  if len(captions) < captions_per_image:
      captions += [random.choice(captions) for _ in range(captions_per_image - len(captions))]
  else:
      captions = random.sample(captions, k=captions_per_image)
    
  # sanity check
  assert len(captions) == captions_per_image

  # encode and pad captions
  enc_captions = []
  caption_lens = []
  for idx, caption in enumerate(captions):
      enc_caption = [word2id['<start>']] + [word2id.get(word, word2id['<unk>']) for word in caption] + [
                      word2id['<end>']] + [word2id['<pad>']] * (max_len - len(caption))
        
      # find caption length (including start and end token)
      caption_len = len(caption) + 2

      enc_captions.append(enc_caption)
      caption_lens.append(caption_len)

  # get the absolute path of image
  path = image_folder_train + 'VizWiz_train_' + '%08d.jpg' % (val['image_id'])
  #print(path)

  # add the image path, encoded caption to the list
  if split=="train":
      train_image_paths.append(path)
      train_image_captions.append(enc_captions)
      train_caption_lens.append(caption_lens)
      #print(path," ", enc_captions, " ", caption_lens)
    
# sanity check
assert len(train_image_paths) == len(train_image_captions)

In [ ]:
val_image_paths = []
val_image_captions = []
val_caption_lens = []

split = "val"
count = 0
captions = []
for val in data_val['annotations']:
  #print(val)
  path = image_folder_val + 'VizWiz_val_' + '%08d.jpg' % (val['image_id']-23431)
  count = count+1
  caption = val['caption']
  #print(caption)
  caption_words = caption.split()
  for i in range(len(caption_words)):
    word = caption_words[i]
    word = word.lower()
    caption_words[i] = word
  if(len(caption_words) <= max_len):
    captions.append(caption_words)
    
  if(len(captions) == 0):
    continue
    
  # sample the captions randomly such that every image has same no of captions
  if len(captions) < captions_per_image:
      captions += [random.choice(captions) for _ in range(captions_per_image - len(captions))]
  else:
      captions = random.sample(captions, k=captions_per_image)
    
  # sanity check
  assert len(captions) == captions_per_image

  # encode and pad captions
  enc_captions = []
  caption_lens = []
  for idx, caption in enumerate(captions):
      enc_caption = [word2id['<start>']] + [word2id.get(word, word2id['<unk>']) for word in caption] + [
                      word2id['<end>']] + [word2id['<pad>']] * (max_len - len(caption))
        
      # find caption length (including start and end token)
      caption_len = len(caption) + 2

      enc_captions.append(enc_caption)
      caption_lens.append(caption_len)

  # get the absolute path of image
  path = image_folder_val + 'VizWiz_val_' + '%08d.jpg' % (val['image_id']-23431)
  #print(path)

  # add the image path, encoded caption to the list
  if split=="val":
      val_image_paths.append(path)
      val_image_captions.append(enc_captions)
      val_caption_lens.append(caption_lens)
      #print(path," ", enc_captions, " ", caption_lens)
    
# sanity check
assert len(val_image_paths) == len(val_image_captions)

In [ ]:
# sets device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class CaptionDataset(Dataset):
    def __init__(self, image_paths, captions, caption_lens, split, captions_per_image=5, transform=None):
        self.split = split
        assert self.split in {'TRAIN', 'VAL', 'TEST'}

        self.image_paths = image_paths
        self.captions = captions
        self.caption_lens = caption_lens
        self.captions_per_image = captions_per_image

        # pytorch transformation pipeline for image (normalizing, etc)
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths) * self.captions_per_image
    
    def __getitem__(self, idx):
        # remember the Nth caption belongs to (N // captions_per_image)th image
        img_path = (self.image_paths[idx//self.captions_per_image])
        img = Image.open(self.image_paths[idx // self.captions_per_image]).convert('RGB')
        img = np.array(img)
        img = cv2.resize(img, (256, 256))

        # sanity check
        assert img.shape == (256, 256, 3)
        assert np.max(img) <= 255.

        if self.transform is not None:
            img = self.transform(img)
        
        img = np.array(img)
        img = np.transpose(img, (2, 0, 1))
        img = torch.FloatTensor(img / 255.)

        caption = torch.LongTensor(self.captions[idx // self.captions_per_image][idx % self.captions_per_image])
        caption_len = torch.LongTensor([self.caption_lens[idx // self.captions_per_image][idx % self.captions_per_image]])

        if self.split == 'TRAIN':
            #print("train-20")
            return img, caption, caption_len, img_path
        else:
            # return all captions per image for validation or testing to find the BLEU-4 scores
            all_captions = torch.LongTensor(self.captions[idx // self.captions_per_image])
            return img, caption, caption_len, img_path, all_captions

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])
train_dataset = CaptionDataset(train_image_paths, train_image_captions, train_caption_lens, 'TRAIN')
val_dataset = CaptionDataset(val_image_paths, val_image_captions, val_caption_lens, 'VAL')

In [ ]:
batch_size = 1
workers = 2

# dataloader for train and val set
train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                          shuffle=True, num_workers=workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, 
                          shuffle=True, num_workers=workers, pin_memory=True)

# quick check to see if data is properly loaded or not
images, captions, caption_lens, img_paths = next(iter(train_loader))
print(images.shape)
print(captions.shape)
print(caption_lens.shape)
print(len(img_paths))

torch.Size([1, 3, 256, 256])
torch.Size([1, 102])
torch.Size([1, 1])
1


In [ ]:
def decode_caption(enc_caption):
    dec_caption = [
        id2word[id]
        for id in caption.numpy()
        if id2word[id] not in ["<start>", "<end>", "<unk>", "<pad>"]
    ]
    return " ".join(dec_caption)

In [ ]:
class ImageEncoder(nn.Module):
    def __init__(self, enc_image_size=14):
        super(ImageEncoder, self).__init__()
        self.enc_image_size = enc_image_size

        # pretrained ImageNet ResNeXt-101
        resnext = models.resnext101_32x8d(pretrained=True)

        # remove linear and pool layers
        modules = list(resnext.children())[:-2]
        #print(modules)
        self.resnext = nn.Sequential(*modules)

        # resize image to fixed size using adaptive pool to allow input images of variable size
        #self.adaptive_pool = nn.AdaptiveAvgPool2d((enc_image_size, enc_image_size))

        self.fine_tune()

    def fine_tune(self, fine_tune=True):
        """
        Allow or prevent computation of the gradients for convolutional blocks 2 through 4 of the image encoder.
        :param fine_tune: boolean
        """
        for param in self.resnext.parameters():
            param.requires_grad = False
        # if fine-tuning, fine-tune convolutional blocks 2 through 4
        for child in list(self.resnext.children())[5:]:
            for param in child.parameters():
                param.requires_grad = fine_tune
    
    def forward(self, images): #, img_paths, split
        """
        Forward propagation.
        :param images: images, a tensor of dim (batch_size, 3, image_size, image_size)
        :return enc_images: encoded repr of images, a tensor of dim (batch_size, enc_image_size, enc_image_size, 2048)
        """
        out = self.resnext(images)       # (batch_size, 2048, image_size/32, image_size/32)
        #out = self.adaptive_pool(out)   # (batch_size, 2048, enc_image_size, enc_image_size)
        out = out.permute(0, 2, 3, 1) 
          # (batch_size, enc_image_size, enc_image_size, 2048)
        #print(out.size())

        """
        out = out.detach().to("cpu").numpy()
        features_out = []
        for i in range(len(out)):
          o = out[i]
          o_changed = np.reshape(o, (1,2048,14,14))
          #print(o_changed.shape)
          features_out.append(o_changed)
        print(features_out[0].shape)
        print(len(features_out))

        word_array_det, features_det = yolo_obj_det(img_paths)
        word_array_ocr, features_ocr = ocr(split)
        
        features_inter = []
        for i in range(len(features_out)):
          inter = np.empty((1,2348,14,14))
          inter[:,:2048] = features_out[i]
          inter[:,2048:,:,:] = features_ocr[i]
          #print(inter.shape)
          features_inter.append(inter)
          #print(inter)
        print(len(features_inter))


        features_result = []
        for i in range(len(features_inter)):
          result = np.empty((1,2648,14,14))
          result[:,:2348] = features_inter[i]
          result[:,2348:,:,:] = features_det[i]
          #print(result.shape)
          #print(result)
          features_result.append(result)
        print(len(features_result))

        for i in range(len(features_result)):
          result = np.transpose(features_result[i], [0,2,3,1])
          features_result[i] = result

        print(features_result[0].shape)

        features_encoded = np.vstack((features_result))
        print(features_encoded.shape)
        
        out_final = features_encoded #(batch_size,14,14,2648)
        out_final = torch.from_numpy(out_final).to("cuda")
        out_final = out_final.float()
        """

        return out

In [ ]:
class Attention(nn.Module):
    def __init__(self, encoder_dim, decoder_dim, attention_dim):
        """
        :param encoder_dim: feature size of encoded images
        :param decoder_dim: size of decoder's RNN
        :param attention_dim: size of the attention network
        """
        super(Attention, self).__init__()
        self.encoder_attn = nn.Linear(encoder_dim, attention_dim)   # linear layer to transform encoder's output
        self.decoder_attn = nn.Linear(decoder_dim, attention_dim)   # linear layer to transform decoder's output
        self.full_attn = nn.Linear(attention_dim, 1)
    
    def forward(self, encoder_out, decoder_hidden):
        """
        Forward propagation (uses Multiplicative attention).
        :param encoder_out: encoded images, a tensor of dim (batch_size, num_pixels, encoder_dim)
        :param decoder_hidden: previous decoder output, a tensor of dim (batch_size, decoder_dim)
        """
        attn1 = self.encoder_attn(encoder_out)          # (batch_size, num_pixels, attention_dim)
        attn2 = self.decoder_attn(decoder_hidden)       # (batch_size, attention_dim)
        attn = self.full_attn(F.relu(attn1 + attn2.unsqueeze(1)))    # (batch_size, num_pixels, 1)

        # apply softmax to calculate weights for weighted encoding based on attention
        alpha = F.softmax(attn, dim=1)                  # (batch_size, num_pixels, 1)
        attn_weighted_encoding = (encoder_out * alpha).sum(dim=1)  # (batch_size, encoder_dim)
        alpha = alpha.squeeze(2)  # (batch_size, num_pixels)
        return attn_weighted_encoding, alpha

In [ ]:
class DecoderWithAttention(nn.Module):
    def __init__(self, attention_dim, embed_dim, decoder_dim, vocab_size, encoder_dim=2048, dropout=0.5):
        """
        :param attention_dim: size of attention network
        :param embed_dim: embedding_size
        :param decoder_dim: feature size of decoder's RNN
        :param vocab_size: size of vocabulary
        :param encoder_dim: feature size of encoded images
        :param dropout: dropout
        """
        super(DecoderWithAttention, self).__init__()
        self.encoder_dim = encoder_dim
        self.attention_dim = attention_dim
        self.embed_dim = embed_dim
        self.decoder_dim = decoder_dim
        self.vocab_size = vocab_size
        self.dropout = dropout

        self.attention = Attention(encoder_dim, decoder_dim, attention_dim)     # attention network
        self.embedding = nn.Embedding(vocab_size, embed_dim)                    # embedding layer
        self.dropout = nn.Dropout(p=dropout)

        self.decode_step = nn.LSTMCell(embed_dim + encoder_dim, decoder_dim, bias=True) # decoding LSTMCell
        self.init_h = nn.Linear(encoder_dim, decoder_dim)   # linear layer to find initial hidden state of LSTMCell
        self.init_c = nn.Linear(encoder_dim, decoder_dim)    # linear layer to find initial cell state of LSTMCell
        self.f_beta = nn.Linear(decoder_dim, encoder_dim)   # linear layer to create a sigmoid-activated gate

        self.fc = nn.Linear(decoder_dim, vocab_size)        # linear layer to find scores over vocabulary
        
        self.init_weights()     # initialize some layers with the uniform distribution
    
    def init_weights(self):
        """
        Initialize some layers with the uniform distribution for easier convergence.
        """
        self.embedding.weight.data.uniform_(-0.1, 0.1)
        self.fc.bias.data.fill_(0)
        self.fc.weight.data.uniform_(-0.1, 0.1)
    
    def init_hidden_state(self, encoder_out):
        """
        Creates the initial hidden and cell states for the decoder's LSTM based on the encoded images.
        :param encoder_out: encoded_images, a tensor of dimension (batch_size, num_pixels, encoder_dim)
        :return hidden_state, cell_state
        """
        mean_encoder_out = encoder_out.mean(dim=1)  # (batch_size, encoder_dim)
        h = self.init_h(mean_encoder_out)   # (batch_size, decoder_dim)
        c = self.init_c(mean_encoder_out)   # (batch_size, decoder_dim)
        return h, c
    
    def forward(self, encoder_out, encoded_captions, caption_lens):
        """
        Forward propagation.
        :param encoder_out: encoded images, a tensor of dimension (batch_size, enc_image_size, enc_image_size, encoder_dim)
        :param encoded_captions: encoded captions, a tensor of dimension (batch_size, max_caption_length)
        :param caption_lengths: caption lengths, a tensor of dimension (batch_size, 1)
        :return: scores for vocabulary, sorted encoded captions, decode lengths, weights, sort indices
        """
        batch_size = encoder_out.size(0)
        #print("batch size in decoder: ",batch_size)
        # flatten image
        encoder_out = encoder_out.view(batch_size, -1, 2048)     # (batch_size, num_pixels, encoder_dim)
        #print("encoder_out.size(): ",encoder_out.size())
        num_pixels = encoder_out.size(1)
        
        # sort the input data by the decreasing caption length
        caption_lens, sort_idx = caption_lens.squeeze(1).sort(dim=0, descending=True)
        encoder_out = encoder_out[sort_idx]
        encoded_captions = encoded_captions[sort_idx]

        # embedding
        embeddings = self.embedding(encoded_captions)   # (batch_size, max_caption_length, embed_dim)

        # initialize lstm state
        h, c = self.init_hidden_state(encoder_out)      # (batch_size, decoder_dim)

        # We won't decode at the <end> position, since we've finished generating as soon as we generate <end>
        # So, decoding lengths are caption lengths - 1
        decode_lens = (caption_lens - 1).tolist()

        # create tensors to hold word prediction scores and alphas
        predictions = torch.zeros(batch_size, max(decode_lens), vocab_size).to(device)
        alphas = torch.zeros(batch_size, max(decode_lens), num_pixels).to(device)

        # At each time-step, decode by attention-weighing the encoder's output based on the 
        # decoder's previous hidden state output then generate a new word in the decoder with 
        # the previous word and the attention weighted encoding
        for t in range(max(decode_lens)):
            # get the batch size for each time step t
            batch_size_t = sum([l > t for l in decode_lens])

            # get the attention weighted encodings (batch_size_t, encoder_dim)
            attention_weighted_encoding, alpha = self.attention(encoder_out[:batch_size_t], h[:batch_size_t])

            gate = F.sigmoid(self.f_beta(h[:batch_size_t]))     # sigmoid gating scalar, (batch_size_t, encoder_dim)
            attention_weighted_encoding = gate * attention_weighted_encoding

            # get the decoder hidden state and cell state based on the embeddings of timestep t word 
            # and the attention weighted encoding
            h, c = self.decode_step(
                torch.cat([embeddings[:batch_size_t, t, :], attention_weighted_encoding], dim=1),
                (h[:batch_size_t], c[:batch_size_t])
            )   # (batch_size_t, decoder_dim)

            # get the next word prediction
            preds = self.fc(self.dropout(h))    # (batch_size_t, vocab_size)

            # save the prediction and alpha for every time step
            predictions[:batch_size_t, t, :] = preds
            alphas[:batch_size_t, t, :] = alpha
    
        return predictions, encoded_captions, decode_lens, alphas, sort_idx


In [ ]:
def generate_image_caption(encoder, decoder, image_path, word_map, beam_size=5):
 
    # Reads an image and captions it with beam search as well as plot attention maps.

    i = Image.open(image_path)
    print(i)
    k = beam_size
    
    # id to word mapping
    rev_word_map = {id: word for word, id in word_map.items()}

    # read and pre-process image
    img = np.array(Image.open(image_path).convert('RGB'))
    img = np.array(Image.open(image_path).convert('RGB'))
    img = cv2.resize(img, (256, 256))

    # sanity check
    assert img.shape == (256, 256, 3)
    assert np.max(img) <= 255

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    img = transform(img)    # (3, 256, 256)

    # encode the image
    encoder_out = encoder(img.unsqueeze(0).to(device))     # (1, enc_image_size, enc_image_size, encoder_dim)
    enc_image_size = encoder_out.size(1)
    encoder_dim = encoder_out.size(3)

    # flatten encoded image representation
    encoder_out = encoder_out.view(1, -1, encoder_dim)  # (1, num_pixels, encoder_dim)
    num_pixels = encoder_out.size(1)

    # we'll treat the problem as having a batch size of k
    encoder_out = encoder_out.expand(k, num_pixels, encoder_dim)    # (k, num_pixels, encoder_dim)

    # tensor to store top k previous words at each step; now they're just <start>
    top_k_prev_words = torch.tensor([[word_map['<start>']]] * k, dtype=torch.long).to(device)   # (k, 1)

    # tensor to store top k sequences; now they're just <start>
    top_k_seqs = top_k_prev_words   # (k, 1)

    # tensor to store top k sequences' scores; now they're just 0
    top_k_scores = torch.zeros(k, 1).to(device)     # (k, 1)

    # tensor to store top k sequences' alphas; now they're just 1s
    top_k_seqs_alpha = torch.ones(k, 1, enc_image_size, enc_image_size).to(device)  # (k, 1, enc_image_size, enc_image_size)

    # lists to store completed sequences along with their alphas and scores
    complete_seqs = []
    complete_seqs_alpha = []
    complete_seqs_scores = []

    # start decoding
    step = 1
    h, c = decoder.init_hidden_state(encoder_out)

    # s is a number less than or equal to k, because sequences are removed from this process once they hit <end>
    while True:
        embeddings = decoder.embedding(top_k_prev_words).squeeze(1)  # (s, embed_dim)

        attention_weighted_encoding, alpha = decoder.attention(encoder_out, h)  # (s, encoder_dim), (s, num_pixels, 1)
        alpha = alpha.view(-1, enc_image_size, enc_image_size)     # (s, enc_image_size, enc_image_size)

        gate = F.sigmoid(decoder.f_beta(h))      # gating scalar, (s, encoder_dim)
        attention_weighted_encoding = gate * attention_weighted_encoding

        h, c = decoder.decode_step(
            torch.cat([embeddings, attention_weighted_encoding], dim=1), (h, c)
        )   # (s, decoder_dim)

        scores = decoder.fc(h)      # (s, vocab_size)
        scores = F.log_softmax(scores, dim=1)

        # add the scores to prev scores
        scores = top_k_scores.expand_as(scores) + scores    # (s, vocab_size)

        # all the k points will have the same score for the first step (since same k previous words, h, c)
        if step == 1:
            top_k_scores, top_k_words = scores[0].topk(k, 0, True, True)    # (s)
        else:
            # unroll and find top scores, and their unrolled indices
            top_k_scores, top_k_words = scores.view(-1).topk(k, 0, True, True)    # (s)
        
        # convert unrolled indices to actual indices of scores
        prev_word_inds = top_k_words // vocab_size  # (s)
        next_word_inds = top_k_words % vocab_size  # (s)

        # add new words to sequences, alphas
        top_k_seqs = torch.cat([top_k_seqs[prev_word_inds], next_word_inds.unsqueeze(1)], dim=1)    # (s, step + 1)
        top_k_seqs_alpha = torch.cat(
            [top_k_seqs_alpha[prev_word_inds], alpha[prev_word_inds].unsqueeze(1)], dim=1
        )   # (s, step + 1, enc_image_size, enc_image_size)
        
        # which sequences are incomplete (didn't reach <end>)?
        incomplete_inds = [ind for ind, next_word in enumerate(next_word_inds) 
                            if next_word != word_map['<end>']]
        complete_inds = list(set(range(len(next_word_inds))) - set(incomplete_inds))

        # set aside complete sequences
        if len(complete_inds) > 0:
            complete_seqs.extend(top_k_seqs[complete_inds].tolist())
            complete_seqs_alpha.extend(top_k_seqs_alpha[complete_inds].tolist())
            complete_seqs_scores.extend(top_k_scores[complete_inds])
        k -= len(complete_inds)  # reduce beam length accordingly

        # proceed with incomplete sequences
        if k == 0:
            break
        
        top_k_seqs = top_k_seqs[incomplete_inds]
        top_k_seqs_alpha = top_k_seqs_alpha[incomplete_inds]
        h = h[prev_word_inds[incomplete_inds]]
        c = c[prev_word_inds[incomplete_inds]]
        encoder_out = encoder_out[prev_word_inds[incomplete_inds]]
        top_k_scores = top_k_scores[incomplete_inds].unsqueeze(1)
        top_k_prev_words = next_word_inds[incomplete_inds].unsqueeze(1)

        # break if things have been going on too long
        if step > 50:
            break
        step += 1
    
    # select sequence with max score
    i = complete_seqs_scores.index(max(complete_seqs_scores))
    seq = complete_seqs[i]
    alphas = complete_seqs_alpha[i]

    caption = [rev_word_map[ind] for ind in seq]

    img = Image.open(image_path).convert('RGB')
    img = img.resize([14 * 24, 14 * 24], Image.LANCZOS)

    # plot the image and attention map
    fig = plt.figure(figsize=(20, 8))
    for t in range(len(caption)):
        
        plt.subplot(np.ceil(len(caption) / 5.), 5, t + 1)
        plt.text(0, 1, '%s' % (caption[t]), color='black', backgroundcolor='white', fontsize=12)
        plt.imshow(img)

        current_alpha = alphas[t]
        alpha = skimage.transform.pyramid_expand(current_alpha, upscale=24, sigma=8)
        if t == 0:
            plt.imshow(alpha, alpha=0)
        else:
            plt.imshow(alpha, alpha=0.8)
        
        plt.set_cmap('gray')
        plt.axis('off')

    plt.show()

In [ ]:
#@title Generate Caption for an Image

image_url = 'https://drive.google.com/drive/folders/1vOgmwqJiDyPyOF5vymc9sQQKDklO2kb7/' #@param {type:"string"}
beam_size =  5 #@param {type: "number"}

checkpoint = torch.load('/content/drive/My Drive/Final Year Project/image_captioning_cnn_best.pth', map_location=device)

encoder = checkpoint['encoder']
decoder = checkpoint['decoder']

vocab_size = len(word2id)
image_path = '/content/drive/My Drive/Final Year Project/dataset/test/VizWiz_test_00000019.jpg'

from urllib.request import urlretrieve
#urlretrieve(image_url, image_path)

generate_image_caption(encoder, decoder, image_path, word2id, beam_size)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
checkpoint = torch.load('/content/drive/My Drive/Final Year Project/image_captioning_cnn_best.pth', map_location=device)
encoder = checkpoint['encoder']
decoder = checkpoint['decoder']
bleu = checkpoint['bleu-4']
epochs = checkpoint['epoch']
epochs_since_improvement = checkpoint['epochs_since_improvement']

In [ ]:
fine_tune_encoder = False  # fine-tune encoder?
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning

encoder = ImageEncoder()
encoder.fine_tune(fine_tune_encoder)
encoder_optimizer = optim.Adam(params=filter(lambda p: p.requires_grad, encoder.parameters()), lr=encoder_lr) if fine_tune_encoder else None

Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth


  0%|          | 0.00/340M [00:00<?, ?B/s]

In [ ]:
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 758 kB/s 


In [ ]:
import onnx
import torch
from PIL import Image

In [ ]:
img = Image.open('/content/drive/My Drive/Final Year Project/dataset/train/VizWiz_train_00015692.jpg')


In [ ]:
import torchvision.transforms as transforms

resize = transforms.Resize([224, 224])
img = resize(img)


to_tensor = transforms.ToTensor()
img = to_tensor(img)
img.unsqueeze_(0)

out = encoder(img)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
example_input = img.to(device)
pytorch_model = encoder.to(device)
ONNX_PATH="./my_model.onnx"



In [ ]:
images = encoder(images.to(device))

In [ ]:
# model parameters
embed_dim = 512      # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512    # dimension of decoder RNN
encoder_dim = 2048
vocab_size = len(word2id)

In [ ]:

decoder = DecoderWithAttention(attention_dim, embed_dim, decoder_dim, vocab_size, encoder_dim)

In [ ]:
pytorch_model_decoder = decoder.to(device)
ONNX_PATH_decoder="./decoder.onnx"

In [ ]:
torch.onnx.export(
    model=pytorch_model,
    args=example_input, 
    f=ONNX_PATH, # where should it be saved
    verbose=False,
    export_params=True,
    do_constant_folding=False,  # fold constant values for optimization
    # do_constant_folding=True,   # fold constant values for optimization
    input_names=['input'],
    output_names=['output'],
   
   
)

In [ ]:
onnx_model = onnx.load(ONNX_PATH)
onnx.checker.check_model(onnx_model)

In [ ]:
!pip install onnx_tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 226 kB 10.1 MB/s 
     |████████████████████████████████| 1.1 MB 66.2 MB/s 


In [ ]:
from onnx_tf.backend import prepare
import onnx

TF_PATH = "./my_tf_model.pb" # where the representation of tensorflow model will be stored
ONNX_PATH = "./my_model.onnx" # path to my existing ONNX model
onnx_model = onnx.load(ONNX_PATH)  # load onnx model
tf_rep = prepare(onnx_model)
tf_rep.export_graph(TF_PATH)

INFO:tensorflow:Assets written to: ./my_tf_model.pb/assets


INFO:tensorflow:Assets written to: ./my_tf_model.pb/assets


In [ ]:
TF_PATH = "./my_tf_model.pb" # where the forzen graph is stored
TFLITE_PATH = "./my_model.tflite"

In [ ]:
os.environ["PATH"] = "C:\\Users\\Hp\\anaconda3\\envs\\tensorflow\\Library\\bin"

In [ ]:
import tensorflow as tf

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(TF_PATH)

In [ ]:
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]

In [ ]:
tf_lite_model = converter.convert()
# Save the model.
with open(TFLITE_PATH, 'wb') as f:
    f.write(tf_lite_model)